In [2]:
import sys
print(f"{sys.version}")

3.9.4 (default, Apr  9 2021, 16:34:09) 
[GCC 7.3.0]


In [3]:
from enum import Enum
from typing import List, Tuple, NamedTuple, Callable, Optional

Grid = List[List[int]]
Options = List[List[List[int]]]

GRID_SIZE: int = 9
SECTOR_SIZE: int = 3
VALUE_RANGE: int = 9
EMPTY = 0

class GridLocation(NamedTuple):
    row: int
    column: int



In [4]:
# sample puzzles
puz1 = [[5,1,7,6,0,0,0,3,4],
         [2,8,9,0,0,4,0,0,0],
         [3,4,6,2,0,5,0,9,0],
         [6,0,2,0,0,0,0,1,0],
         [0,3,8,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inp2  = [[5,1,7,6,0,0,0,3,4],
         [0,8,9,0,0,4,0,0,0],
         [3,0,6,2,0,5,0,9,0],
         [6,0,0,0,0,0,0,1,0],
         [0,3,0,0,0,6,0,4,7],
         [0,0,0,0,0,0,0,0,0],
         [0,9,0,0,0,0,0,7,8],
         [7,0,3,4,0,0,5,6,0],
         [0,0,0,0,0,0,0,0,0]]

inpd  = [[1,0,5,7,0,2,6,3,8],
         [2,0,0,0,0,6,0,0,5],
         [0,6,3,8,4,0,2,1,0],
         [0,5,9,2,0,1,3,8,0],
         [0,0,2,0,5,8,0,0,9],
         [7,1,0,0,3,0,5,0,2],
         [0,0,4,5,6,0,7,2,0],
         [5,0,0,0,0,4,0,6,3],
         [3,2,6,1,0,7,0,0,4]]

hard  = [[8,5,0,0,0,2,4,0,0],
         [7,2,0,0,0,0,0,0,9],
         [0,0,4,0,0,0,0,0,0],
         [0,0,0,1,0,7,0,0,2],
         [3,0,5,0,0,0,9,0,0],
         [0,4,0,0,0,0,0,0,0],
         [0,0,0,0,8,0,0,7,0],
         [0,1,7,0,0,0,0,0,0],
         [0,0,0,0,3,6,0,4,0]]

diff  = [[0,0,5,3,0,0,0,0,0],
         [8,0,0,0,0,0,0,2,0],
         [0,7,0,0,1,0,5,0,0],
         [4,0,0,0,0,5,3,0,0],
         [0,1,0,0,7,0,0,0,6],
         [0,0,3,2,0,0,0,8,0],
         [0,6,0,5,0,0,0,0,9],
         [0,0,4,0,0,0,0,3,0],
         [0,0,0,0,0,9,7,0,0]]


In [5]:
class Sudoku():
    def __init__(self, initial_values: Grid = None, rows: int = GRID_SIZE, columns: int = GRID_SIZE ) -> None:
        self._rows: int = rows
        self._columns: int = columns
        if initial_values == None:
            self.grid: Grid = [[EMPTY for c in range(columns)] for r in range(rows)]
        elif len(initial_values) != rows:
            print(f"initial_values: {len(initial_values)} rows, should be {GRID_SIZE}.")
        elif not (all([len(r)==GRID_SIZE for r in initial_values])):
            print(f"initial_values: not all rows contain {GRID_SIZE} columns.")
        else:
            self.grid: Grid = initial_values
        self._options: Options = self.get_options()


    def __str__(self) -> str:
        first_row = True
        output: str = "["
        for row in self.grid:
            first_col = True
            if first_row:
                output += "["
                first_row = False
            else:
                output += " ["
            for col in row:
                if first_col:
                    output += str(col)
                    first_col = False
                else:
                    output += ", " + str(col)
            output += "],\n"
        output += "]\n"
        return output


    def _init_domain(self) -> Options:
        """
        initializes a list array of 1 through 9 domain options for each cell
        """
        domain = []
        for r in range(self._rows):
            row = []
            for c in range(self._columns):
                row.append(list(range(1,VALUE_RANGE+1)))
            domain.append(row)
        return domain


    def get_options(self) -> Options:
        """
        given a sudoku puzzle grid with values entered, an array of options for each cell is returned  
        """
        opts = self._init_domain()
        for r in range(self._rows):
            for c in range(self._columns):
                if self.grid[r][c] != EMPTY:
                    opts[r][c] = []
                else:
                    # check row
                    for j in range(self._columns):
                        if self.grid[r][j] in opts[r][c]:
                            opts[r][c].remove(self.grid[r][j])
                    # check column
                    for i in range(self._rows):
                        if self.grid[i][c] in opts[r][c]:
                            opts[r][c].remove(self.grid[i][c])
                    # check sector
                    sec_r = r // SECTOR_SIZE
                    sec_c = c // SECTOR_SIZE
                    for i in range(SECTOR_SIZE):
                        for j in range(SECTOR_SIZE):
                            si = SECTOR_SIZE*sec_r + i
                            sj = SECTOR_SIZE*sec_c + j
                            if self.grid[si][sj] in opts[r][c]:
                                opts[r][c].remove(self.grid[si][sj])
        return opts


    def set_cell(self, rcv: Optional[Tuple[int, int, int]] ) -> bool:
        """
        set a sudoku puzzle grid cell (r, c) to value (v); returns new grid?
        """
        if rcv:
            r, c, v = rcv
            self.grid[r][c] = v
            self._options: Options = self.get_options()
            return True
        else:
            return False


    def print_num_options(self):
        """
        prints a grid conaining number of options for each cell
        """
        for r in range(self._rows):
            print([len(self._options[r][c]) for c in range(self._columns)])


    def num_options_all_zero(self) -> bool:
        """
        if any option counts greater than zero, returns False
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if len(self._options[r][c]) > 0:
                    return False
        return True


    def get_first_single_option(self) -> Optional[Tuple[int, int, int]]:
        """
        find first cell with only one option
            return row , column and the single option
        otherwise return None
        """
        for r in range(self._rows):
            for c in range(self._columns):
                if len(self._options[r][c]) == 1:
                    return (r, c, self._options[r][c][0])
        return None


    def fill_easy(self):
        d = True
        step_count = 0
        while (d):
            d = self.set_cell(self.get_first_single_option())
            step_count += 1
            print(f"Step: {step_count}\n{self}")
        return self.grid


In [6]:
p1 = Sudoku(puz1)

In [7]:
p1

In [8]:
print(p1)

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]



In [9]:
p1.grid[1][2]

9

In [10]:
p1._rows

9

In [11]:
p1._columns

9

In [12]:
p1.grid

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 0, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [13]:
p1._options

[[[], [], [], [], [8, 9], [8, 9], [2, 8], [], []],
 [[], [], [], [1, 3, 7], [1, 3, 7], [], [1, 6, 7], [5], [1, 5, 6]],
 [[], [], [], [], [1, 7, 8], [], [1, 7, 8], [], [1]],
 [[],
  [5, 7],
  [],
  [3, 5, 7, 8, 9],
  [3, 4, 5, 7, 8, 9],
  [3, 7, 8, 9],
  [3, 8, 9],
  [],
  [3, 5, 9]],
 [[1, 9], [], [], [1, 5, 9], [1, 2, 5, 9], [], [2, 9], [], []],
 [[1, 4, 9],
  [5, 7],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 4, 5, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [2, 3, 6, 8, 9],
  [2, 5, 8],
  [2, 3, 5, 6, 9]],
 [[1, 4],
  [],
  [1, 4, 5],
  [1, 3, 5],
  [1, 2, 3, 5, 6],
  [1, 2, 3],
  [1, 2, 3, 4],
  [],
  []],
 [[], [2], [], [], [1, 2, 8, 9], [1, 2, 8, 9], [], [], [1, 2, 9]],
 [[1, 4, 8],
  [2, 5, 6],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 5, 6, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [1, 2, 3, 4, 9],
  [2],
  [1, 2, 3, 9]]]

In [14]:
p1.print_num_options()

[0, 0, 0, 0, 2, 2, 2, 0, 0]
[0, 0, 0, 3, 3, 0, 3, 1, 3]
[0, 0, 0, 0, 3, 0, 3, 0, 1]
[0, 2, 0, 5, 6, 4, 3, 0, 3]
[2, 0, 0, 3, 4, 0, 2, 0, 0]
[3, 2, 3, 6, 8, 6, 5, 3, 5]
[2, 0, 3, 3, 5, 3, 4, 0, 0]
[0, 1, 0, 0, 4, 4, 0, 0, 3]
[3, 3, 3, 6, 8, 6, 5, 1, 4]


In [15]:
p1.num_options_all_zero()

False

In [16]:
p1.get_first_single_option()

(1, 7, 5)

In [17]:
p1.set_cell(p1.get_first_single_option())

True

In [18]:
p1.grid

[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 5, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 0],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [19]:
p1._options

[[[], [], [], [], [8, 9], [8, 9], [2, 8], [], []],
 [[], [], [], [1, 3, 7], [1, 3, 7], [], [1, 6, 7], [], [1, 6]],
 [[], [], [], [], [1, 7, 8], [], [1, 7, 8], [], [1]],
 [[],
  [5, 7],
  [],
  [3, 5, 7, 8, 9],
  [3, 4, 5, 7, 8, 9],
  [3, 7, 8, 9],
  [3, 8, 9],
  [],
  [3, 5, 9]],
 [[1, 9], [], [], [1, 5, 9], [1, 2, 5, 9], [], [2, 9], [], []],
 [[1, 4, 9],
  [5, 7],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 4, 5, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [2, 3, 6, 8, 9],
  [2, 8],
  [2, 3, 5, 6, 9]],
 [[1, 4],
  [],
  [1, 4, 5],
  [1, 3, 5],
  [1, 2, 3, 5, 6],
  [1, 2, 3],
  [1, 2, 3, 4],
  [],
  []],
 [[], [2], [], [], [1, 2, 8, 9], [1, 2, 8, 9], [], [], [1, 2, 9]],
 [[1, 4, 8],
  [2, 5, 6],
  [1, 4, 5],
  [1, 3, 5, 7, 8, 9],
  [1, 2, 3, 5, 6, 7, 8, 9],
  [1, 2, 3, 7, 8, 9],
  [1, 2, 3, 4, 9],
  [2],
  [1, 2, 3, 9]]]

In [20]:
p1.print_num_options()

[0, 0, 0, 0, 2, 2, 2, 0, 0]
[0, 0, 0, 3, 3, 0, 3, 0, 2]
[0, 0, 0, 0, 3, 0, 3, 0, 1]
[0, 2, 0, 5, 6, 4, 3, 0, 3]
[2, 0, 0, 3, 4, 0, 2, 0, 0]
[3, 2, 3, 6, 8, 6, 5, 2, 5]
[2, 0, 3, 3, 5, 3, 4, 0, 0]
[0, 1, 0, 0, 4, 4, 0, 0, 3]
[3, 3, 3, 6, 8, 6, 5, 1, 4]


In [21]:
p1.fill_easy()

Step: 1
[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 5, 0],
 [3, 4, 6, 2, 0, 5, 0, 9, 1],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]

Step: 2
[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 0, 5, 6],
 [3, 4, 6, 2, 0, 5, 0, 9, 1],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]

Step: 3
[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 7, 5, 6],
 [3, 4, 6, 2, 0, 5, 0, 9, 1],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 0, 0, 7, 8],
 [7, 0, 3, 4, 0, 0, 5, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
]

Step: 4
[[5, 1, 7, 6, 0, 0, 0, 3, 4],
 [2, 8, 9, 0, 0, 4, 7, 5, 6],
 [3, 4, 6, 2, 0, 5, 8, 9, 1],
 [6, 0, 2, 0, 0, 0, 0, 1, 0],
 [0, 3, 8, 0, 0, 6, 0, 4, 7],

[[5, 1, 7, 6, 9, 8, 2, 3, 4],
 [2, 8, 9, 1, 3, 4, 7, 5, 6],
 [3, 4, 6, 2, 7, 5, 8, 9, 1],
 [6, 7, 2, 8, 4, 9, 3, 1, 5],
 [1, 3, 8, 5, 2, 6, 9, 4, 7],
 [9, 5, 4, 7, 1, 3, 6, 8, 2],
 [4, 9, 5, 3, 6, 2, 1, 7, 8],
 [7, 2, 3, 4, 8, 1, 5, 6, 9],
 [8, 6, 1, 9, 5, 7, 4, 2, 3]]

In [22]:
p1.num_options_all_zero()

True

In [23]:
p2 = Sudoku(inp2)